# HierarchicalMultiLabelClassifier
## Create and evaluate a Hierarchical Multi-Label Classifier

# Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
# # Credit: https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
import data
from tensorflow import keras
from loaders import PremadeTripletClassifierSequence
import numpy as np
import pandas as pd
from keras import backend as K
import sys

/home/transposons/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Files

In [4]:
data_dir = '../Data/Datasets'

fantom_path = f"{data_dir}/Fantom"
all_path = f"{data_dir}/All"
train_fantom_path = f"{fantom_path}/train.usage.matrix"
valid_fantom_path = f"{fantom_path}/valid.usage.matrix"
test_fantom_path = f"{fantom_path}/test.usage.matrix"

train_sequence = f"{all_path}/train_sequences.fa"
valid_sequence = f"{all_path}/valid_sequences.fa"
test_sequence = f"{all_path}/test_sequences.fa"

reverse_train_sequence = f'{all_path}/reverse_train_sequences.fa'
reverse_valid_sequence = f'{all_path}/reverse_valid_sequences.fa'
reverse_test_sequence = f'{all_path}/reverse_test_sequences.fa'

train_triplet_dis = f'{all_path}/train_triplet_dis.npy'
valid_triplet_dis = f'{all_path}/valid_triplet_dis.npy'
test_triplet_dis = f'{all_path}/test_triplet_dis.npy'

train_triplet_sim = f'{all_path}/train_triplet_sim.npy'
valid_triplet_sim = f'{all_path}/valid_triplet_sim.npy'
test_triplet_sim = f'{all_path}/test_triplet_sim.npy'

## Parameters

In [5]:
codings_size = 100 # Number of neurons for dense layers
exp_filter_num = 8 # Number of filters for convolutional layers
exp_filter_1d_size = 11 # Filter Size for 1D convolutional layers
max_len = 600 # Maximum length of sequence
allow_reverse = True # Allow reverse complement?
threshold = 0.55 # threshold for classifier

## Data

In [6]:
def make_x_y_z_w(fp, sp):
    '''
    fp: path to the FANTOM matrix file
    sp: path to the FASTA file with every enhancer and random sequence
    
    Returns x, y, z, and w:
    x: the one hot encodings of the enhancers and random sequences (including the reverse complement of each)
    y: contains the target data for the multi-label (in the form of tissues that the enhancer is active in)
    z: contains the binary labels (in the form of, "Is the sequence an enhancer?")
    w: contains the sample weights of each sequence. The enhancers are given a separate weight from the random sequences, which are given a weight of 1.
    '''
    
    
    # m contains the FANTOM enhancer matrix
    m = pd.read_csv(fp, sep = '\t').to_numpy()[:, 1:]
    # x contains the enhancers and random sequences
    x = data.FantomToOneHotConverterRCAddition(sp, 0, 600).seq_matrix

    # y contains the multi-labels for acive tissue
    y = np.zeros((len(x), m.shape[-1]), dtype = np.ubyte)
    
    # z contains binary labels for enhancer or not
    z = np.zeros((len(x), 1), dtype=np.ubyte)
    
    # Sample weights. A weight must be an integer because of the data type
    w = np.ones((len(x), 1), dtype=np.ubyte)
    
    seq_size = len(x) // 2

    # Accounting for reverse complement: Original - RC - Original - RC
    for i in range(len(m)):
        y[i, :] = m[i, :]
        z[i, 0] = 1
        w[i, 0] = 12
        
    for i in range(seq_size, len(m) + seq_size):
        y[i, :] = m[i - seq_size, :]
        z[i, 0] = 1
        w[i, 0] = 12
    
    return np.expand_dims(x, axis=3), y, z, w

In [7]:
train_x, train_y, train_z, train_w = make_x_y_z_w(train_fantom_path, train_sequence)
valid_x , valid_y , valid_z , valid_w = make_x_y_z_w(valid_fantom_path, valid_sequence)

In [9]:
def randomize(x, y, z, w):
    indices = np.random.permutation(len(x))
    
    return x[indices], y[indices], z[indices], w[indices]

In [10]:
d1, d2, d3 = train_x.shape[1:]
print(d1, d2, d3)

4 600 1


In [13]:
train_x, train_y, train_z, train_w = randomize(train_x, train_y, train_z, train_w)
valid_x, valid_y, valid_z, valid_w = randomize(valid_x, valid_y, valid_z, valid_w)

## Creating Hierarchical Classifier Model

In [18]:
model = nets.make_multi_conv_classifier(codings_size, (d1, d2, d3), exp_filter_1d_size, filter_num=exp_filter_num, kernel_2d_col = 3, class_num = train_y.shape[1])

### Creating Loss and Metric Functions

In [20]:
def jaccard_index_ones(y_true, y_pred):
    '''
    Custom metric that uses Jaccard index for positives only
    '''
    # Applying a threshold to get the binary predictions
    y_pred = tf.cast(tf.math.greater(y_pred, tf.constant([0.5])), dtype=tf.float32)
    y_true = tf.cast(y_true, dtype=tf.float32)

    # Calculating the intersection
    intersection = tf.math.reduce_sum(y_true * y_pred, axis=-1)

    # Calculating the union
    union = tf.math.reduce_sum(y_true + y_pred, axis=-1) - intersection

    
    return intersection / (union + K.epsilon())

In [21]:
def jaccard_index_zeros(y_true, y_pred):
    '''
    Custom metric that uses Jaccard index for negatives only
    '''
    y_pred = tf.cast(tf.math.greater(y_pred, tf.constant([0.5])), dtype=tf.float32)
    y_pred = 1 - y_pred
    y_true = 1 - y_true
    
    return jaccard_index_ones(y_true, y_pred)

In [22]:
def jaccard_index_loss(y_true, y_pred):
    '''
    Custom loss function that calculates the jaccard index loss
    '''
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    y_true = tf.cast(y_true, dtype=tf.float32)

    # Calculating the intersection
    intersection = tf.math.reduce_sum(y_true * y_pred, axis=-1)

    # Calculating the union
    union = tf.math.reduce_sum(y_true + y_pred, axis=-1) - intersection

    
    return 1.0 - (intersection / (union + K.epsilon()))

In [23]:
def jaccard_index_f1(y_true, y_pred):
    '''
    Custom loss function that uses the harmonic mean of the Jaccard index (ones and zeros) for positives and negatives
    '''
    negatives = jaccard_index_zeros(y_true, y_pred)
    positives = jaccard_index_ones(y_true, y_pred)
    
    return (2 * negatives * positives) / (negatives + positives + K.epsilon())

In [24]:
def recall(y_true, y_pred):
    '''
    Custom metric that calculates recall
    '''
    # Applying a threshold to get the binary predictions
    y_pred = tf.cast(tf.math.greater(y_pred, tf.constant([0.5])), dtype=tf.float32)
    y_true = tf.cast(y_true, dtype=tf.float32)

    # Calculating the True Positives
    true_positives = tf.math.reduce_sum(y_true * y_pred, axis=-1)

    # Calculating the possible positives
    all_positives = tf.math.reduce_sum(y_true, axis=-1)

    # Calculating Recall
    recall_value = true_positives / (all_positives + K.epsilon())

    return recall_value

In [27]:
def specificity(y_true, y_pred):
    '''
    Custom metric that calculates specificity
    '''
    # Applying a threshold to get the binary predictions
    y_pred = tf.cast(tf.math.greater(y_pred, tf.constant([0.5])), dtype=tf.float32)
    y_pred = 1 - y_pred
    y_true = 1 - y_true
    
    return recall(y_true, y_pred)

In [28]:
def accuracy(y_true, y_pred):
    '''
    Custom metric that calculates accuracy
    '''
    y_pred = tf.cast(tf.math.greater(y_pred, tf.constant([0.5])), dtype = tf.float32)
    y_true = tf.cast(y_true, dtype = tf.float32)

    correct_labels = tf.cast(tf.math.equal(y_true, y_pred), dtype = tf.float32)
    return tf.math.reduce_mean(correct_labels, axis = -1)

### Compiling

In [31]:
opt = keras.optimizers.SGD(learning_rate=1.0, momentum=0.9, nesterov=True) # 0.1

In [32]:

model.compile(loss={'output_1':jaccard_index_loss, 'output_2':'mse'},
              metrics={'output_1':[jaccard_index_f1, recall, specificity, accuracy], 'output_2':[tf.keras.metrics.AUC(curve="ROC"), tf.keras.metrics.Recall(name="binary_recall"), tf.keras.metrics.Precision(name="binary_precision")]}, 
              optimizer=opt)

In [33]:
early_stopping = keras.callbacks.EarlyStopping(patience=20, min_delta=1/100000, restore_best_weights=True, monitor='val_loss', start_from_epoch = 10, mode='min') 

In [34]:
def exponential_decay_fn(epoch, lr):
    '''
    This function decreases the learning rate according to the epoch
    '''
    return lr*0.1**(1/100)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

### Training

In [41]:
model.fit(x=train_x, y=[train_y, train_z], epochs=500, batch_size =1024, validation_data=(valid_x, [valid_y, valid_z]), workers=26, 
          callbacks=[early_stopping, lr_scheduler],sample_weight=pd.Series(np.squeeze(train_w)).to_frame('wt')) 

Epoch 1/500
975/975 [==============================] - 69s 62ms/step - loss: 2.3574 - output_1_loss: 1.7950 - output_2_loss: 0.5624 - output_1_jaccard_index_f1: 0.0033 - output_1_recall: 0.0449 - output_1_specificity: 0.5156 - output_1_accuracy: 0.5157 - output_2_auc: 0.5763 - output_2_binary_recall: 0.6028 - output_2_binary_precision: 0.0945 - val_loss: 1.2898 - val_output_1_loss: 0.9977 - val_output_2_loss: 0.2920 - val_output_1_jaccard_index_f1: 0.0048 - val_output_1_recall: 0.0473 - val_output_1_specificity: 0.6652 - val_output_1_accuracy: 0.6651 - val_output_2_auc: 0.5968 - val_output_2_binary_recall: 0.6857 - val_output_2_binary_precision: 0.0909 - lr: 0.9772
Epoch 2/500
975/975 [==============================] - 60s 62ms/step - loss: 2.1310 - output_1_loss: 1.7764 - output_2_loss: 0.3546 - output_1_jaccard_index_f1: 0.0061 - output_1_recall: 0.0374 - output_1_specificity: 0.6030 - output_1_accuracy: 0.6024 - output_2_auc: 0.7693 - output_2_binary_recall: 0.6980 - output_2_binary

In [47]:
model.save("MultiLabelAux")

INFO:tensorflow:Assets written to: MultiLabelAux/assets


INFO:tensorflow:Assets written to: MultiLabelAux/assets


## Triplet Data

In [49]:
train = data.FantomToOneHotConverter(train_sequence, 0, max_len).seq_matrix
reverse_train = data.FantomToOneHotConverter(reverse_train_sequence, 0, max_len).seq_matrix

In [50]:
valid = data.FantomToOneHotConverter(valid_sequence, 0, max_len).seq_matrix
reverse_valid = data.FantomToOneHotConverter(reverse_valid_sequence, 0, max_len).seq_matrix

In [51]:
test = data.FantomToOneHotConverter(test_sequence, 0, max_len).seq_matrix
reverse_test = data.FantomToOneHotConverter(reverse_test_sequence, 0, max_len).seq_matrix

## Loading Datasets

In [52]:
train_seq = PremadeTripletClassifierSequence(train, train_triplet_sim, train_triplet_dis, batch_size = 1024, reverse_x_in = reverse_train if allow_reverse else None)
valid_seq = PremadeTripletClassifierSequence(valid, valid_triplet_sim, valid_triplet_dis, batch_size = 1024, reverse_x_in = reverse_valid if allow_reverse else None)
test_seq = PremadeTripletClassifierSequence(test, test_triplet_sim, test_triplet_dis, batch_size = 1024, reverse_x_in = reverse_test if allow_reverse else None)

In [53]:
custom_objects = {'jaccard_index_loss': jaccard_index_loss,'jaccard_index_f1': jaccard_index_f1, 'recall': recall, 'specificity': specificity}

In [25]:
model = tf.keras.models.load_model("MultiLabelAux", custom_objects = custom_objects) # Loading model

In [54]:
def collect_data_from_loader(a_loader):
    x_list = []
    y_list = []
    for x_batch, y_batch in a_loader:
        x_list.append(x_batch)
        y_list.append(y_batch)
    x_matrix = np.concatenate(x_list)
    y_array  = np.concatenate(y_list).reshape(-1)
    return x_matrix, y_array

In [55]:
x_train, y_train = collect_data_from_loader(train_seq)
x_valid, y_valid = collect_data_from_loader(valid_seq)
x_test, y_test = collect_data_from_loader(test_seq)

## Evaluating Hierarichal Classifier

In [ ]:
# Evaluating Hierarchical Classifier as is 
print("Evaluating training set")
model.evaluate(x=train_x, y=[train_y, train_z], batch_size=1024)

print("Evaluating validation set")
model.evaluate(x=valid_x, y=[valid_y, valid_z], batch_size=1024)

In [56]:
def collect_output(x, model, batch_size):
    '''
    Collecting predictions from a model
    '''
    pred = np.zeros(len(x), dtype = np.ubyte)

    for i in range(0, len(x), batch_size):
        top = min(i + batch_size, len(x))
        padding_size = (i + batch_size) - top

        anc = np.pad(x[i:top, :, :, 0], ((0, padding_size), (0, 0), (0, 0)), mode='constant')
        pos = np.pad(x[i:top, :, :, 1], ((0, padding_size), (0, 0), (0, 0)), mode='constant')
        neg_sim = np.pad(x[i:top, :, :, 2], ((0, padding_size), (0, 0), (0, 0)), mode='constant')
        
        anc_tissues, _  = model.predict(anc)
        pos_tissues, _  = model.predict(pos)
        neg_sim_tissues, is_enhancers = model.predict(neg_sim)
        
        anc_tissues = tf.math.round(anc_tissues)
        pos_tissues = tf.math.round(pos_tissues)
        neg_sim_tissues = tf.math.round(neg_sim_tissues)
        is_enhancers = tf.math.round(is_enhancers)
        
        end = batch_size if padding_size == 0 else len(x) - i
        for j in range(end):
            if is_enhancers[j, 0] == 0:
                pred[j] = 0
            else:
                is_sharing = len(np.intersect1d(anc_tissues[j], neg_sim_tissues[j])) > 0 and len(np.intersect1d(pos_tissues[j], neg_sim_tissues[j])) > 0
                pred[j] = 1 if is_sharing else 0
                        
    return pred

In [58]:
def calculate_metrics(y, pred):
    '''
    Calculating metrics for a model
    '''
    
    accuracy = nets.crm_accuracy(y, pred)
    specificity = nets.crm_specificity(y, pred)
    recall = nets.crm_recall(y, pred)
    precision = nets.crm_precision(y, pred)
    f1 = nets.crm_f1_score(y, pred)
    
    print("Accuracy:", accuracy)
    print("Specificity:", specificity)
    print("Recall: ", recall )
    print("Precision: ", precision )
    print("F1: ",  f1)
    
    print("TP: ", tp)
    print("FP: ", fp)
    print("TN: ", tn)
    print("FN: ", fn)
    print("Length: ", len(y))

    r = [accuracy, specificity, recall, precision, f1]

    return r

In [59]:
def evaluate(x, y, model, batch_size):
    '''
    Evaluating a model
    '''
    pred = collect_output(x, model, batch_size)
    return calculate_metrics(y, pred)

In [60]:
r_train = evaluate(x_train, y_train, model, 2048)
r_valid = evaluate(x_valid, y_valid, model, 2048)
r_test = evaluate(x_test, y_test, model, 2048)

64/64 [==============================] - 0s 2ms/step
Accuracy: 0.4999186197916667
Specificity: 0.9934888768312534
Recall:  0.00645546273190843
Precision:  0.497907949790795
F1:  0.012745675574358692
TP:  238.0
FP:  240.0
TN:  36620.0
FN:  36630.0
Length:  73728
64/64 [==============================] - 0s 1ms/step
Accuracy: 0.49830729166666665
Specificity: 0.968961919666145
Recall:  0.029121164846593862
Precision:  0.48484848484848486
F1:  0.05494235957812114
TP:  224.0
FP:  238.0
TN:  7430.0
FN:  7468.0
Length:  15360
64/64 [==============================] - 0s 2ms/step
Accuracy: 0.5008463541666667
Specificity: 0.970504158004158
Recall:  0.029227557411273485
Precision:  0.49667405764966743
F1:  0.05520640788662968
TP:  224.0
FP:  227.0
TN:  7469.0
FN:  7440.0
Length:  15360
